### Feature engineering phase 1

- Remove unnecessary columns to reduce dataframe size
- Filter train records according to min/max langtitude and longitude in grid info
- Generate grid info and aggregate records accroding to VIN and grid id

In [28]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import timeit
%matplotlib inline

# loading training files
ecar_training_data_path = 'data/training/ecar/'
rcar_training_data_path = 'data/training/rcar/'

ecar_output_filepath = 'data/training/ecar/processed/ecar_training.csv'
rcar_output_filepath = 'data/training/rcar/processed/rcar_training.csv'

ecar_filenames = [f for f in os.listdir(ecar_training_data_path)]
rcar_filenames = [f for f in os.listdir(rcar_training_data_path)]

# define drop columns
ecar_drop_columns = ['work_mode','mileage','avg_fuel_consumption','system_mode']
rcar_drop_columns = ['power_mode','mileage','fuel_consumption']
common_drop_columns = ['lat','lon','date_time']

# loading grid min/max lantitude and longitude
grids = pd.read_csv('data/grid_info.csv')
grid_max_lantitude = grids['latitude_to'].max()
grid_min_lantitude = grids['latitude_from'].min()
grid_max_longitude = grids['longitude_to'].max()
grid_min_longitude = grids['longitude_from'].min()

In [26]:
def round_down_lat_lon(df):
    df['lat'] = df.lat.round(3)
    df['lon'] = df.lon.round(3)
    return df
        
def filter_data_by_lan_lon(df):
    df = df[(df.lat >= grid_min_lantitude ) & (df.lat <= grid_max_lantitude)]
    df = df[(df.lon >= grid_min_longitude ) & (df.lon <= grid_max_longitude)]
    return df

def add_grid_id(df):
    df_with_grid_id = pd.DataFrame(index=[0], columns=['car_id', 'date_time','speed','lat','lon','grid_id'])
    df_with_grid_id = df_with_grid_id.fillna(-1)
    for i in range(len(grids)):
        min_lan, max_lan = grids['latitude_from'][i], grids['latitude_to'][i]
        min_lon, max_lon, grid_id = grids['longitude_from'][i], grids['longitude_to'][i], grids['grid_id'][i]
        df_ = df[(df.lat >= min_lan ) & (df.lat <= max_lan)]
        df_ = df_[(df_.lon >= min_lon ) & (df_.lon <= max_lon)]
        df_['grid_id'] = grid_id
        df_with_grid_id = df_with_grid_id.append(df_)
    return df_with_grid_id

def format_date(date_time):
    return date_time[:-9].replace('-','')

def retrieve_hour(date_time):
    return date_time[-8:-6]

def preprocess_data(dataType='rcar'):
    # init run time
    start = timeit.default_timer()
    # define common variables
    filenames = rcar_filenames
    filepath = rcar_training_data_path
    drop_columns = rcar_drop_columns
    output_filepath = rcar_output_filepath    
    
    if dataType == 'ecar':
        filenames = ecar_filenames
        filepath = ecar_training_data_path
        drop_columns = ecar_drop_columns
        output_filepath = ecar_output_filepath
        
    # if file is already exist return
    if os.path.exists(output_filepath):
        print(output_filepath + " already existed, will return directly")
        return
    
    df_ = pd.DataFrame(index=[0], columns=['car_id', 'date','hour','speed', 'grid_id'])
    df_ = df_.fillna(-1)
    for filename in filenames:
        # if is directory skip
        if os.path.isdir(filepath + filename):
            continue
        # loading data
        df = pd.read_csv(filepath + filename, low_memory=False)
        # drop uncessary columns
        df = df.drop(columns=drop_columns)
        # filter data by lat and lon
        df = filter_data_by_lan_lon(df)
        # format data_time column
        df['hour'] = df['date_time'].apply(lambda x : retrieve_hour(x))
        df['date'] = df['date_time'].apply(lambda x : format_date(x))
        # generate new column grid_id
        df = add_grid_id(df)
        # filter data by grid_id
        df = df[df.grid_id != -1]
        # drop date_time, lat and lon
        df = df.drop(columns=common_drop_columns)
        # group by below column and get average speed for each record
        df = df.groupby(['car_id', 'date','hour','grid_id'], as_index=False).mean()
        # round average speed
        df.speed = df.speed.round(3)
        # appending df
        df_ = df_.append(df)
        # remove grid_id = -1
        df_ = df_[df_.grid_id != -1]
    # print log and total time
    print("Finish process all the file, total cost:[", timeit.default_timer() - start ,"] seconds.")
    # write final file
    df_.to_csv(output_filepath)

In [15]:
preprocess_data('rcar')

Finish all the file, total cost:[ 344.8129758159416 ]


In [27]:
preprocess_data('ecar')

data/training/ecar/processed/ecar_training.csv already existed, will return directly


### Feature engineering phase 2
- Merge ecar and rcar training data
- Drop column speed and car_id first(these 2 columns maybe used in future but let's try a simple solution first)
- Aggregate training data and calculate total number of cars per day/per hour/per grid
- Merge training data with weather data

In [57]:
def preprocess_phase1_data():
    # loading files
    ecar = pd.read_csv(ecar_output_filepath)
    rcar = pd.read_csv(rcar_output_filepath)
    # merge data
    df = ecar.append(rcar)
    # remove unnamed column
    df = df.loc[:, ~df.columns.str.contains('^Unnamed')]
    # drop car id and speed
    df = df.drop(columns=['car_id','speed'])
    # add column count
    df['count'] = 1
    # aggregate data by date,hour and grid id
    df = df.groupby(['date','hour','grid_id'], as_index=False).sum()
    # remove unecessary row
    df = df[(df.date != 20161230 ) & (df.date != 20170101)] 
    return df

In [43]:
ecar = pd.read_csv(ecar_output_filepath)
rcar = pd.read_csv(rcar_output_filepath)


# merge data
df = ecar.append(rcar)
df = df.loc[:, ~df.columns.str.contains('^Unnamed')]
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 704240 entries, 0 to 85547
Data columns (total 5 columns):
car_id     704240 non-null object
date       704240 non-null int64
grid_id    704240 non-null int64
hour       704240 non-null int64
speed      702797 non-null float64
dtypes: float64(1), int64(3), object(1)
memory usage: 32.2+ MB


In [44]:
df.head(10)

,car_id,date,grid_id,hour,speed
0,e10041,20170102,15,7,10.919
1,e10041,20170102,19,8,10.324
2,e10041,20170102,36,8,31.252
3,e10041,20170102,38,8,47.213
4,e10041,20170102,43,8,48.924
5,e10041,20170102,19,9,14.548
6,e10041,20170102,42,9,23.282
7,e10041,20170102,43,9,3.901
8,e10041,20170102,24,10,48.054
9,e10041,20170102,26,10,65.844


In [45]:
df = df.drop(columns=['car_id','speed'])

In [46]:
df.head(10)

,date,grid_id,hour
0,20170102,15,7
1,20170102,19,8
2,20170102,36,8
3,20170102,38,8
4,20170102,43,8
5,20170102,19,9
6,20170102,42,9
7,20170102,43,9
8,20170102,24,10
9,20170102,26,10


In [47]:
df['count'] = 1
#df = df.groupby(['date','hour','grid_id'], as_index=False).sum()

In [48]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 704240 entries, 0 to 85547
Data columns (total 4 columns):
date       704240 non-null int64
grid_id    704240 non-null int64
hour       704240 non-null int64
count      704240 non-null int64
dtypes: int64(4)
memory usage: 26.9 MB


In [49]:
df.head()

,date,grid_id,hour,count
0,20170102,15,7,1
1,20170102,19,8,1
2,20170102,36,8,1
3,20170102,38,8,1
4,20170102,43,8,1


In [50]:
df = df.groupby(['date','hour','grid_id'], as_index=False).sum()
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 77614 entries, 0 to 77613
Data columns (total 4 columns):
date       77614 non-null int64
hour       77614 non-null int64
grid_id    77614 non-null int64
count      77614 non-null int64
dtypes: int64(4)
memory usage: 3.0 MB


In [54]:
df.head(20)

,date,hour,grid_id,count
0,20161230,7,41,1
1,20161230,7,44,1
2,20161230,20,33,1
3,20161230,20,41,1
4,20161230,20,44,1
5,20170101,9,15,1
6,20170101,18,42,1
7,20170101,20,15,1
8,20170101,22,19,1
9,20170101,23,15,1


In [55]:
df = df[(df.date != 20161230 ) & (df.date != 20170101)]

df.head(10)

,date,hour,grid_id,count
14,20170102,0,2,1
15,20170102,0,4,5
16,20170102,0,5,3
17,20170102,0,6,3
18,20170102,0,7,1
19,20170102,0,8,5
20,20170102,0,10,5
21,20170102,0,11,4
22,20170102,0,12,5
23,20170102,0,14,1


In [56]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 77600 entries, 14 to 77613
Data columns (total 4 columns):
date       77600 non-null int64
hour       77600 non-null int64
grid_id    77600 non-null int64
count      77600 non-null int64
dtypes: int64(4)
memory usage: 3.0 MB


In [58]:
df__ = preprocess_phase1_data()
df__.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 77600 entries, 14 to 77613
Data columns (total 4 columns):
date       77600 non-null int64
hour       77600 non-null int64
grid_id    77600 non-null int64
count      77600 non-null int64
dtypes: int64(4)
memory usage: 3.0 MB


In [59]:
df__.head(20)

,date,hour,grid_id,count
14,20170102,0,2,1
15,20170102,0,4,5
16,20170102,0,5,3
17,20170102,0,6,3
18,20170102,0,7,1
19,20170102,0,8,5
20,20170102,0,10,5
21,20170102,0,11,4
22,20170102,0,12,5
23,20170102,0,14,1
